In [1]:
import pandas as pd
import time

start = time.time()
df = pd.read_csv("Hotel_Reviews.csv")
end = time.time()
print("Loading took " + str(round(end - start, 2)) + " seconds")

Loading took 3.45 seconds


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Hotel_Address                               515738 non-null  object 
 1   Additional_Number_of_Scoring                515738 non-null  int64  
 2   Review_Date                                 515738 non-null  object 
 3   Average_Score                               515738 non-null  float64
 4   Hotel_Name                                  515738 non-null  object 
 5   Reviewer_Nationality                        515738 non-null  object 
 6   Negative_Review                             515738 non-null  object 
 7   Review_Total_Negative_Word_Counts           515738 non-null  int64  
 8   Total_Number_of_Reviews                     515738 non-null  int64  
 9   Positive_Review                             515738 non-null  object 
 

1. Print out the shape of the data frame you have just loaded (the shape is the number of rows and columns)

In [3]:
df.shape

(515738, 17)

2. Calculate the frequency count for reviewer nationalities:

    i. How many distinct values are there for the column `Reviewer_Nationality` and what are they?
    
    ii. What reviewer nationality is the most common in the dataset (print country and number of reviews)?
    
    iii. What are the next top 10 most frequently found nationalities, and their frequency count?

In [4]:
df["Reviewer_Nationality"].value_counts().head(10)

 United Kingdom               245246
 United States of America      35437
 Australia                     21686
 Ireland                       14827
 United Arab Emirates          10235
 Saudi Arabia                   8951
 Netherlands                    8772
 Switzerland                    8678
 Germany                        7941
 Canada                         7894
Name: Reviewer_Nationality, dtype: int64

3. What was the most frequently reviewed hotel for each of the top 10 most reviewer nationalities?

In [5]:
for nat in df["Reviewer_Nationality"].value_counts().head(10).index:
    nat_df = df[df["Reviewer_Nationality"] == nat]
    freq = nat_df["Hotel_Name"].value_counts()
    print(f"{str(nat)}: {freq.index[0]} - {freq[0]}")

 United Kingdom : Britannia International Hotel Canary Wharf - 3833
 United States of America : Hotel Esther a - 423
 Australia : Park Plaza Westminster Bridge London - 167
 Ireland : Copthorne Tara Hotel London Kensington - 239
 United Arab Emirates : Millennium Hotel London Knightsbridge - 129
 Saudi Arabia : The Cumberland A Guoman Hotel - 142
 Netherlands : Jaz Amsterdam - 97
 Switzerland : Hotel Da Vinci - 97
 Germany : Hotel Da Vinci - 86
 Canada : St James Court A Taj Hotel London - 61


4. How many reviews are there per hotel (frequency count of hotel) in the dataset?

In [6]:
hotel_freq_df = df.drop(
    [
        "Hotel_Address",
        "Additional_Number_of_Scoring",
        "Review_Date",
        "Average_Score",
        "Reviewer_Nationality",
        "Negative_Review",
        "Review_Total_Negative_Word_Counts",
        "Positive_Review",
        "Review_Total_Positive_Word_Counts",
        "Total_Number_of_Reviews_Reviewer_Has_Given",
        "Reviewer_Score",
        "Tags",
        "days_since_review",
        "lat",
        "lng",
    ],
    axis=1,
)
hotel_freq_df["Total_Reviews_Found"] = hotel_freq_df.groupby("Hotel_Name").transform(
    "count"
)
hotel_freq_df = hotel_freq_df.drop_duplicates(subset=["Hotel_Name"])
hotel_freq_df

,Hotel_Name,Total_Number_of_Reviews,Total_Reviews_Found
0,Hotel Arena,1403,405
405,K K Hotel George,1831,566
971,Apex Temple Court Hotel,2619,1037
2008,The Park Grand London Paddington,4380,1770
3778,Monhotel Lounge SPA,171,35
...,...,...,...
511962,Suite Hotel 900 m zur Oper,3461,439
512401,Hotel Amadeus,717,144
512545,The Berkeley,232,100
512645,Holiday Inn London Kensington,5945,2768


5. While there is an `Average_Score` column for each hotel in the dataset, you can also calculate an average score (getting the average of all reviewer scores in the dataset for each hotel). Add a new column to your dataframe with the column header `Calc_Average_Score` that contains that calculated average.

In [7]:
df["Calc_Average_Score"] = round(
    df.groupby("Hotel_Name").Reviewer_Score.transform("mean"), 1
)

In [8]:
df[["Hotel_Name", "Average_Score", "Calc_Average_Score"]].head()

,Hotel_Name,Average_Score,Calc_Average_Score
0,Hotel Arena,7.7,7.8
1,Hotel Arena,7.7,7.8
2,Hotel Arena,7.7,7.8
3,Hotel Arena,7.7,7.8
4,Hotel Arena,7.7,7.8


In [9]:
def get_difference_review_avg(row):
    return row["Average_Score"] - row["Calc_Average_Score"]


df["Average_Score_Difference"] = df.apply(get_difference_review_avg, axis=1)
df[
    ["Hotel_Name", "Average_Score", "Calc_Average_Score", "Average_Score_Difference"]
].head()

,Hotel_Name,Average_Score,Calc_Average_Score,Average_Score_Difference
0,Hotel Arena,7.7,7.8,-0.1
1,Hotel Arena,7.7,7.8,-0.1
2,Hotel Arena,7.7,7.8,-0.1
3,Hotel Arena,7.7,7.8,-0.1
4,Hotel Arena,7.7,7.8,-0.1


In [10]:
review_scores_df = df.drop_duplicates(subset=["Hotel_Name"])
review_scores_df = review_scores_df.sort_values(by=["Average_Score_Difference"])
review_scores_df[
    ["Average_Score_Difference", "Average_Score", "Calc_Average_Score", "Hotel_Name"]
]

,Average_Score_Difference,Average_Score,Calc_Average_Score,Hotel_Name
495945,-0.8,7.7,8.5,Best Western Hotel Astoria
111027,-0.7,8.8,9.5,Hotel Stendhal Place Vend me Paris MGallery by...
43688,-0.7,7.5,8.2,Mercure Paris Porte d Orleans
178253,-0.7,7.9,8.6,Renaissance Paris Vendome Hotel
218258,-0.5,7.0,7.5,Hotel Royal Elys es
...,...,...,...,...
151416,0.7,7.8,7.1,Best Western Allegro Nation
22189,0.8,7.1,6.3,Holiday Inn Paris Montparnasse Pasteur
250308,0.9,8.6,7.7,MARQUIS Faubourg St Honor Relais Ch teaux
68936,0.9,6.8,5.9,Villa Eugenie


6. Calculate and print out how many rows have column `Negative_Review` values of "No Negative"
7. Calculate and print out how many rows have column `Positive_Review` values of "No Positive"
8. Calculate and print out how many rows have column `Positive_Review` values of "No Positive" and `Negative_Review` values of "No Negative"

In [11]:
# with lambdas:
start = time.time()
no_negative_reviews = df.apply(
    lambda x: True if x["Negative_Review"] == "No Negative" else False, axis=1
)
print(
    "Number of No Negative reviews: "
    + str(len(no_negative_reviews[no_negative_reviews == True].index))
)

no_positive_reviews = df.apply(
    lambda x: True if x["Positive_Review"] == "No Positive" else False, axis=1
)
print(
    "Number of No Positive reviews: "
    + str(len(no_positive_reviews[no_positive_reviews == True].index))
)

both_no_reviews = df.apply(
    lambda x: True
    if x["Negative_Review"] == "No Negative" and x["Positive_Review"] == "No Positive"
    else False,
    axis=1,
)
print(
    "Number of both No Negative and No Positive reviews: "
    + str(len(both_no_reviews[both_no_reviews == True].index))
)
end = time.time()
print("Lambdas took " + str(round(end - start, 2)) + " seconds")

Number of No Negative reviews: 127890
Number of No Positive reviews: 35946
Number of both No Negative and No Positive reviews: 127
Lambdas took 22.22 seconds


In [12]:
# without lambdas (using a mixture of notations to show you can use both)
start = time.time()
no_negative_reviews = sum(df.Negative_Review == "No Negative")
print("Number of No Negative reviews: " + str(no_negative_reviews))

no_positive_reviews = sum(df["Positive_Review"] == "No Positive")
print("Number of No Positive reviews: " + str(no_positive_reviews))

both_no_reviews = sum(
    (df.Negative_Review == "No Negative") & (df.Positive_Review == "No Positive")
)
print("Number of both No Negative and No Positive reviews: " + str(both_no_reviews))

end = time.time()
print("Sum took " + str(round(end - start, 2)) + " seconds")

Number of No Negative reviews: 127890
Number of No Positive reviews: 35946
Number of both No Negative and No Positive reviews: 127
Sum took 0.4 seconds


In [13]:
start = time.time()
df = pd.read_csv("Hotel_Reviews.csv")
end = time.time()

In [14]:
def replace_address(row):
    if "Netherlands" in row["Hotel_Address"]:
        return "Amsterdam, Netherlands"
    elif "Barcelona" in row["Hotel_Address"]:
        return "Barcelona, Spain"
    elif "United Kingdom" in row["Hotel_Address"]:
        return "London, United Kingdom"
    elif "Milan" in row["Hotel_Address"]:
        return "Milan, Italy"
    elif "France" in row["Hotel_Address"]:
        return "Paris, France"
    elif "Vienna" in row["Hotel_Address"]:
        return "Vienna, Austria"


# Replace all the addresses with a shortened, more useful form
df["Hotel_Address"] = df.apply(replace_address, axis=1)
# The sum of the value_counts() should add up to the total number of reviews
print(df["Hotel_Address"].value_counts())

London, United Kingdom    262301
Barcelona, Spain           60149
Paris, France              59928
Amsterdam, Netherlands     57214
Vienna, Austria            38939
Milan, Italy               37207
Name: Hotel_Address, dtype: int64


In [15]:
df.groupby("Hotel_Address").agg({"Hotel_Name": "nunique"})

,Hotel_Name
Hotel_Address,
"Amsterdam, Netherlands",105
"Barcelona, Spain",211
"London, United Kingdom",400
"Milan, Italy",162
"Paris, France",458
"Vienna, Austria",158


In [16]:
# Drop `Additional_Number_of_Scoring`
df.drop(["Additional_Number_of_Scoring"], axis=1, inplace=True)
# Replace `Total_Number_of_Reviews` and `Average_Score` with our own calculated values
df.Total_Number_of_Reviews = df.groupby("Hotel_Name").transform("count")[
    "Hotel_Address"
]  # need to take any one of the cols to map the values
df.Average_Score = round(df.groupby("Hotel_Name").Reviewer_Score.transform("mean"), 1)

In [17]:
# Remove opening and closing brackets
df.Tags = df.Tags.str.strip("[']")
# remove all quotes too
df.Tags = df.Tags.str.replace(" ', '", ",", regex=False)

In [18]:
# Process the Tags into new columns
# The file Hotel_Reviews_Tags.py, identifies the most important tags
# Leisure trip, Couple, Solo traveler, Business trip, Group combined with Travelers with friends,
# Family with young children, Family with older children, With a pet
df["Leisure_trip"] = df.Tags.apply(lambda tag: 1 if "Leisure trip" in tag else 0)
df["Couple"] = df.Tags.apply(lambda tag: 1 if "Couple" in tag else 0)
df["Solo_traveler"] = df.Tags.apply(lambda tag: 1 if "Solo traveler" in tag else 0)
df["Business_trip"] = df.Tags.apply(lambda tag: 1 if "Business trip" in tag else 0)
df["Group"] = df.Tags.apply(
    lambda tag: 1 if "Group" in tag or "Travelers with friends" in tag else 0
)
df["Family_with_young_children"] = df.Tags.apply(
    lambda tag: 1 if "Family with young children" in tag else 0
)
df["Family_with_older_children"] = df.Tags.apply(
    lambda tag: 1 if "Family with older children" in tag else 0
)
df["With_a_pet"] = df.Tags.apply(lambda tag: 1 if "With a pet" in tag else 0)

In [19]:
df.drop(
    [
        "Review_Total_Negative_Word_Counts",
        "Review_Total_Positive_Word_Counts",
        "days_since_review",
        "Total_Number_of_Reviews_Reviewer_Has_Given",
    ],
    axis=1,
    inplace=True,
)

# Saving new data file with calculated columns
print("Saving results to Hotel_Reviews_Filtered.csv")
df.to_csv(r"Hotel_Reviews_Filtered.csv", index=False)

Saving results to Hotel_Reviews_Filtered.csv


In [22]:
import time
import pandas as pd
import nltk as nltk
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download("vader_lexicon")
nltk.download("stopwords")

# Load the filtered hotel reviews from CSV
df = pd.read_csv("Hotel_Reviews_Filtered.csv")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\trung\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\trung\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [23]:
from nltk.corpus import stopwords

start = time.time()
cache = set(stopwords.words("english"))


def remove_stopwords(review):
    text = " ".join([word for word in review.split() if word not in cache])
    return text


# Remove the stop words from both columns
df.Negative_Review = df.Negative_Review.apply(remove_stopwords)
df.Positive_Review = df.Positive_Review.apply(remove_stopwords)

In [24]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Create the vader sentiment analyser (there are others in NLTK you can try too)
vader_sentiment = SentimentIntensityAnalyzer()
# Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text. Eighth International Conference on Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.

# There are 3 possibilities of input for a review:
# It could be "No Negative", in which case, return 0
# It could be "No Positive", in which case, return 0
# It could be a review, in which case calculate the sentiment
def calc_sentiment(review):
    if review == "No Negative" or review == "No Positive":
        return 0
    return vader_sentiment.polarity_scores(review)["compound"]

In [25]:
# Add a negative sentiment and positive sentiment column
print("Calculating sentiment columns for both positive and negative reviews")
start = time.time()
df["Negative_Sentiment"] = df.Negative_Review.apply(calc_sentiment)
df["Positive_Sentiment"] = df.Positive_Review.apply(calc_sentiment)
end = time.time()
print("Calculating sentiment took " + str(round(end - start, 2)) + " seconds")

Calculating sentiment columns for both positive and negative reviews
Calculating sentiment took 207.91 seconds


In [26]:
df = df.sort_values(by=["Negative_Sentiment"], ascending=True)
print(df[["Negative_Review", "Negative_Sentiment"]])
df = df.sort_values(by=["Positive_Sentiment"], ascending=True)
print(df[["Positive_Review", "Positive_Sentiment"]])

                                          Negative_Review  Negative_Sentiment
186584  So bad experience memories I hotel The first n...             -0.9920
129503  First charged twice room booked booking second...             -0.9896
307286  The staff Had bad experience even booking Janu...             -0.9889
201953  Everything DO NOT STAY AT THIS HOTEL I never i...             -0.9886
452092  No WLAN room Incredibly rude restaurant staff ...             -0.9884
...                                                   ...                 ...
138365  Wifi terribly slow I speed test network upload...              0.9938
79215   I find anything hotel first I walked past hote...              0.9938
278506  The property great location There bakery next ...              0.9945
339189  Guys I like hotel I wish return next year Howe...              0.9948
480509  I travel lot far visited countless number hote...              0.9957

[515738 rows x 2 columns]
                                     

In [27]:
df = df.reindex(
    [
        "Hotel_Name",
        "Hotel_Address",
        "Total_Number_of_Reviews",
        "Average_Score",
        "Reviewer_Score",
        "Negative_Sentiment",
        "Positive_Sentiment",
        "Reviewer_Nationality",
        "Leisure_trip",
        "Couple",
        "Solo_traveler",
        "Business_trip",
        "Group",
        "Family_with_young_children",
        "Family_with_older_children",
        "With_a_pet",
        "Negative_Review",
        "Positive_Review",
    ],
    axis=1,
)

print("Saving results to Hotel_Reviews_NLP.csv")
df.to_csv(r"Hotel_Reviews_NLP.csv", index=False)

Saving results to Hotel_Reviews_NLP.csv
